# Pattern 2 — *Post-Addition Attention* (Residual + Attention Fusion)

Bu not, **Pattern 2**’yi “en temel → en ileri” sırayla anlatır ve **Pattern 1** ile net şekilde karşılaştırır.  
Odak: *Attention’ı nereye koyduğun* (fusion), *çarpma-toplama*, *skip güvenliği*, *stabilizasyon*.

---
## İçindekiler
1. Residual blok hatırlatma: `x`, `F(x)` ve `y`
2. Pattern 1 (Inside residual branch) — kısa tekrar
3. Pattern 2 (Post-addition attention) — temel tanım
4. Neden “daha agresif”? Skip güvenliği ne olur?
5. Pattern 2’nin artıları / eksileri (pratik sezgi)
6. Stabil hale getirme yöntemleri (λ, identity-preserving, init)
7. CBAM ile Pattern 2 (channel + spatial) akışı
8. Kod: Pattern 1 vs Pattern 2 block (PyTorch)
9. Hızlı debug checklist (sık hatalar)


## 1) Residual blok hatırlatma: `x`, `F(x)` ve `y`

Bir residual blok iki akıştan oluşur:

- **Skip/identity yolu:** `x` (referans bilgi, kısa yol)
- **Residual yolu:** `F(x)` (modelin öğrendiği “düzeltme/katkı”)

En klasik form:

\[
y = x + F(x)
\]

Burada kritik nokta: Skip yolu sayesinde **gradient yolu açık kalır**. Bu, derin ağlarda eğitim stabilitesinin temelidir.


## 2) Pattern 1 (Inside residual branch) — kısa tekrar

Pattern 1’de attention **sadece** residual yolun ürettiği `F(x)` üzerinde çalışır:

\[
y = x + \big(A(F(x)) \odot F(x)\big)
\]

- Attention’a giren: `F(x)`
- Attention çıktısı: maske/weights `A(F(x))`
- İşlem: **çarpma** (⊙) ile `F(x)` yeniden ağırlıklandırılır
- Skip yolu `x`: **dokunulmaz**, en sonda toplanır

**Sezgi:** “Eklediğim şeyi seç; referansa dokunma.”  
Bu yüzden Pattern 1 genelde **en güvenli default** kabul edilir.


## 3) Pattern 2 (Post-Addition Attention) — temel tanım

Pattern 2’de önce **klasik residual toplaması yapılır**, sonra attention bu toplamın üstüne uygulanır.

Adımlar:

1) Klasik birleşim:
\[
z = x + F(x)
\]

2) Attention, birleşik temsil üzerinde:
\[
y = A(z) \odot z
\]

Yani Pattern 2’nin net formu:

\[
y = A(x + F(x)) \odot (x + F(x))
\]

**Sezgi:** “Önce toplam sonucu oluştur; sonra toplam sonucu seç/filtrele.”

Buradaki büyük fark:
- Pattern 1: attention yalnızca `F(x)`’i etkiler
- Pattern 2: attention **hem** `F(x)` hem `x` (skip katkısı) üstünde etkilidir çünkü ikisi `z` içinde birleşmiştir.


## 4) Neden “daha agresif”? Skip güvenliği ne olur?

Pattern 2’de attention, **skip yolunun katkısını da ölçekler**. Çünkü skip katkısı artık `z` içinde.

- Pattern 1’de `x` her zaman “garanti hat” gibi akar.
- Pattern 2’de `x` doğrudan korunmaz; `A(z)` maskesi `z`’yi kısarsa, skip’in etkisi de kısılır.

Bu yüzden Pattern 2:
- temsil gücünü artırabilir (toplam sonucu seçer)
- ama yanlış yerde/yanlış şekilde kullanılırsa eğitim dinamiğini daha hassas yapabilir.

**Basit düşünce:**  
Pattern 1 = “eklenen kısmı ayarla”  
Pattern 2 = “toplamı ayarla” (skip dahil)


## 5) Pattern 2’nin artıları / eksileri (pratik sezgi)

### Artıları
- **Daha güçlü seçicilik:** Attention, toplam temsilin tamamını düzenler.
- **Gürültü bastırma:** Bazı katmanlarda (özellikle daha üst seviyelerde) birleşik temsil temizlenebilir.
- **Feature yeniden kalibrasyonu:** Sadece residual katkı değil, “nihai blok çıktısı” kalibre edilir.

### Eksileri
- **Skip güvenliği azalır:** `A(z)` küçük değerler öğrenirse, `x` katkısı da azalır.
- **Daha hassas eğitim:** Öğrenme oranı, init, sigmoid saturasyonu gibi detaylara daha duyarlı olabilir.
- **Erken katmanlarda risk:** İlk katmanlarda temsil hamdır; agresif attention erken bilgi kaybına yol açabilir.

Kısa kural:
- Pattern 1: genelde güvenli “default”
- Pattern 2: daha agresif; genelde “later stages” veya kontrollü gating ile daha mantıklı


## 6) Stabil hale getirme yöntemleri (λ, identity-preserving, init)

Pattern 2’yi pratikte “kontrollü” yapmak için birkaç yaygın yaklaşım vardır.

### (a) Öğrenilebilir/ayarlı ölçek (λ) ile karıştırma (mixing)
Önce `z = x + F(x)` sonra:

\[
y = (1-\lambda)\, z + \lambda \, (A(z)\odot z)
\]

- \(\lambda=0\) → tamamen klasik residual (y=z)
- \(\lambda=1\) → tam Pattern 2
- Pratikte \(\lambda\) küçük başlatılabilir (ör. 0.1) veya öğrenilebilir.

Bu yöntem şu anlama gelir:  
> “Attention’a ip bağla; başta az etki etsin, sonra aç.”

### (b) Identity-preserving attention (sapmayı ölçekle)
\[
y = z + \lambda \, (A(z)\odot z - z)
\]

Bu form, “attention uygulanmış hâl” ile “orijinal hâl” arasındaki farkı ölçekler. Yine \(\lambda\) küçük başlatılabilir.

### (c) Initialization ile güvenli başlangıç
- Maskeyi üreten katmanların bias/init ayarı ile `A(z) ≈ 1` başlangıcı hedeflenir.
- Böylece model başta “normal residual gibi” davranır, attention yavaşça öğrenilir.

**Özet:** Pattern 2 kullanılacaksa, çoğu zaman “tam serbest bırakmak” yerine **kontrollü açmak** daha güvenlidir.


## 7) CBAM ile Pattern 2 (channel + spatial) akışı

CBAM iki maskeyi ardışık uygular:
- Channel maskesi: \(\alpha_c \in \mathbb{R}^{C\times 1\times 1}\)
- Spatial maskesi: \(\alpha_s \in \mathbb{R}^{1\times H\times W}\)

Pattern 2’de CBAM’in girdiği şey `z`’dir:

1) Birleşim:
\[
z = x + F(x)
\]

2) Channel attention (z üstünde):
\[
z' = \alpha_c(z) \odot z
\]

3) Spatial attention (z' üstünde):
\[
y = \alpha_s(z') \odot z'
\]

Bu durumda CBAM, skip katkısını da içerdiği için **blok çıkışını komple kalibre etmiş** olur.


## 8) Kod: Pattern 1 vs Pattern 2 block (PyTorch)

Aşağıdaki kod iki bloğu yan yana verir:

- **Pattern 1:** CBAM yalnızca `F(x)` üstünde
- **Pattern 2:** önce `z = x + F(x)`, sonra CBAM `z` üstünde


In [ ]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        hidden = max(channels // reduction, 4)
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.max = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.mlp(self.avg(x)) + self.mlp(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        pad = 3 if k == 7 else 1
        self.conv = nn.Conv2d(2, 1, k, padding=pad, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_map = torch.mean(x, dim=1, keepdim=True)
        max_map, _ = torch.max(x, dim=1, keepdim=True)
        cat = torch.cat([avg_map, max_map], dim=1)
        return self.sigmoid(self.conv(cat))

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16, k=7):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention(k)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

class FxConv(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.act   = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        f = self.act(self.bn1(self.conv1(x)))
        f = self.bn2(self.conv2(f))
        return f

def make_skip(in_ch, out_ch, stride):
    if stride != 1 or in_ch != out_ch:
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
            nn.BatchNorm2d(out_ch),
        )
    return nn.Identity()

class Block_P1(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.F = FxConv(in_ch, out_ch, stride=stride)
        self.skip = make_skip(in_ch, out_ch, stride)
        self.cbam = CBAM(out_ch)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = self.skip(x)
        f = self.F(x)       # F(x)
        f = self.cbam(f)    # A(F(x)) ⊙ F(x)
        y = identity + f
        return self.act(y)

class Block_P2(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.F = FxConv(in_ch, out_ch, stride=stride)
        self.skip = make_skip(in_ch, out_ch, stride)
        self.cbam = CBAM(out_ch)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = self.skip(x)
        f = self.F(x)            # F(x)
        z = identity + f         # z = x + F(x)
        y = self.cbam(z)         # y = A(z) ⊙ z
        return self.act(y)

x = torch.randn(2, 64, 32, 32)
print("P1:", Block_P1(64,64)(x).shape)
print("P2:", Block_P2(64,64)(x).shape)

P1: torch.Size([2, 64, 32, 32])
P2: torch.Size([2, 64, 32, 32])


## 9) Hızlı debug checklist (sık hatalar)

### Pattern 2
- [ ] Önce `z = x + F(x)` var mı?
- [ ] Attention’a giren `z` mi?
- [ ] Attention sonucu **çarpma** ile mi uygulanıyor? (`A(z) ⊙ z`)
- [ ] Boyut/kanal değişiyorsa skip eşitlemesi var mı?

### Pattern 1 vs Pattern 2
- **Pattern 1:** attention `F(x)` üstünde → skip korunur  
- **Pattern 2:** attention `z` üstünde → skip de kalibre edilir

### Stabilizasyon (öneri)
- [ ] Pattern 2’de \(\lambda\) ile karıştırma/ölçek kullanıyor muyum?
- [ ] Başlangıçta `A(z) ≈ 1` olacak init/bias stratejim var mı?

---
## Tek cümlelik fark
- **Pattern 1:** “Sadece residual katkıyı seç; skip’i koru.”  
- **Pattern 2:** “Toplam sonucu seç; skip dahil hepsini kalibre et.”
